#Predicting Friends

###Approach
We search for friends by looking at two degrees of separation (i.e. friends of friends only). We score candidates by number of common friends, then by their overall popularity (i.e. their total number of friends). Underling both selection and scoring criteria we assume a transitive property of friendship. This too can be tested. 

We decompose the problem into several steps. We first create a friend dictionary. The dictionary is a suitable data structure because it allows for quick key-lookup, and we do not require the list to be ordered.
 
In step 2 we create a universe of possible candidates. When we retrieve friend lists we convert them into sets. Sets provide efficient difference operations, and remove duplicates. We identify friends of friends not already our friends.

In step 3 we score the candidates. We consider number of friends in common, and candidate’s total number of friends (as a proxy for popularity, more friends is thought of as better)

We have tried to keep the code lean and readable. For simplicity we have not addressed all use cases, ie. requesting more friend recommendations than within 2-degree circles. We currently return min(5,candidates in universe) candidates. 

Coding and ways of storing data could be optimized as our scoring algorithm grows in sophistication, but also upon integration with existing processes, ie. if a friend dictionary already exists, etc. 

###Improvements
__Universe Selection:__ Candidate universe selection is done only for efficiency; it is time-consuming to run our scoring algorithm on the entire population, so we identify a subset. Ideally if we ran our scoring algorithm on the entire population we should do no better than on our narrower universe.

We can broaden the candidate selection method by considering higher degrees of relational separation. Doing this may yield better results especially when making suggestions for poorly-connected users. Characteristics like location and age group can equally help guide the direction in which to expand our universe.

__Candidate Scoring:__ We could give more weight to relationships that transact together more frequently, and are in the same location. A third-degree connection with whom a second-degree connection often transacts may merit a higher score than a stale second-degree one. Further research & testing would need to inform the degree to which this is true.

We could give more weight to recent transactions; we could also give more weight to newly formed relationships (e.g. a new person joining the social circle). These again are hypotheses we could test.

We can also look at directionality, who added whom first? who initiates payment and request payments more? Are these ‘*initiaters*’ more receptive to finding new friends than passive friend/payment accepters?

Social networks often look like dense clusters joined together by connectors. Helping bridge these clusters could open up entirely new friend circles. Identifying points of traction without seeming arbitrary and haphazard is difficult, but promising.

Popularity as a measure of candidacy could conceivably work in the opposite direction, and for a opposite of reasons – a Candidate B who shares all of our user’s friends and himself has no other friends may be a better potential friend than Candidate C who has all of our user’s friends and 2000 more (or maybe our user already knows Candidate B -- B has no other friends, recall -- and they are in fact enemies, but maybe once Candidate B has at least 20 not-common friends things change …). 

###Summary
*We search for friend candidates by looking at two degrees of seperation (ie. friends of friends only). We 
score candidates by number of common friends, and then by their overall popularity (ie. their total number of friends).*

The model is implemented via several helper functions. 
- 'recommend_friend' takes a user_id and outputs a list of 5 recommendations. It uses the 3 functions below
- 'populate_dict' takes a pair of friends and appends each to the dict item for that user
- 'identify_candidates' takes a user id and a friend dictionary and identifies all connections with two degrees of 
  seperation. We populate a list of friends of friends and remove existing friends to create our universe
- 'score_candidates' accepts a user id a list of candidates and scores each of the candidates with respect to the user.
  We score candidates based on the number of common friends, and their total number of friends

Sample Usage:
recommend_friend(59003)


-----------
###Composite function to calculate friend recommendations

####The function:
- populates a dictionary using pairwise friend combinations
- identifies friend candidates
- scores candidates and returns top 5 recommendations

In [ ]:
def recommend_friend(user_id, recommendations = 5, raw_data=raw_data):
    
    friend_dict = dict()
    map(lambda x,y: populate_dict(x,y,friend_dict),raw_data.ix[:,0], raw_data.ix[:,1])   # populate dictionary
    
    friend_candidates = identify_candidates(user_id,friend_dict)
    
    scored_candidates = score_candidates(user_id, friend_candidates,friend_dict)

    return scored_candidates.sort_values(['common_friends', 'number_friends'], ascending = [0,0])[:recommendations]

-----------
###Populate a dictionary symmetrically with a given pair of values

In [ ]:
def populate_dict(friend_a,friend_b,dict):

    if not friend_a in dict:
        dict[friend_a] = [friend_b]
    else:
        dict[friend_a].append(friend_b)
        
    if not friend_b in dict:
        dict[friend_b] = [friend_a]
    else:
        dict[friend_b].append(friend_a)

-----------
###Identify all first-order connections not currently friends

*Notes:
- if a poorly connected user is selected, or too high a recommendation count is set, the candidate universe may 
be too small. It can be increased by increasing to higher degrees of seperation, or randomly selecting candidates.
This functionality has not been incorporated*

In [ ]:
def identify_candidates(friend_id,friend_dict):
    
    immediate_friends = friend_dict[friend_id]
    
    # populate first-order connections, including duplicates, and self
    friends_of_friends =[]
    for friend in immediate_friends:
        friends_of_friends.extend(friend_dict[friend])

    # remove duplicates, existing friends, and self
    friends_of_friends_Set = set(friends_of_friends)
    immediate_friends_Set = set(immediate_friends)
    unconnected_FoF = friends_of_friends_Set - immediate_friends_Set
    unconnected_FoF.discard(friend_id)

    return list(unconnected_FoF)

-----------
###Score friend candidates based on:
- # of common friends;
- popularity (# of total friends)

In [15]:
def score_candidates(person_id, candidate_list,friend_dict):

    person_a_friends = set(friend_dict[person_id])
    
    def pairwise_score(person_b):

        person_b_friends = set(friend_dict[person_b])
        
        common_friends = len(person_a_friends & person_b_friends)
        person_b_number_friends = len(person_b_friends)
        
        return common_friends, person_b_number_friends

    candidates = pd.DataFrame(candidate_list, columns={'candidate_id'})
    candidates['common_friends'], candidates['number_friends'] = zip(*candidates['candidate_id'].map(pairwise_score))
              
    return candidates

-----------
###Sample Output

In [16]:
import pandas as pd
raw_data = pd.read_csv('raw_data.csv', header=0)
recommend_friend(59003)

,candidate_id,common_friends,number_friends
12267,65105,40,113
11668,63576,37,132
7019,50312,36,140
8682,55135,34,238
4879,45619,32,135
